# Running Individual Pipeline Steps

This notebook walks through calibrating the data with individual pipeline steps rather than running the entire pipeline stage.

**Use case**: When using a package outside of the standard JWST pipeline, there may be certain steps that are still helpful to utilize within the JWST pipeline.<br>
**Data**: JWST/NIRISS images and spectra from program 2079.<br>
**Tools**: astropy, crds, glob, jdaviz, jwst, matplotlib, numpy, os, pandas, warnings<br>
**Cross-instrument**: NIRISS<br>

**Content**
- [Imports & Data Setup](#imports)
- [Running Individual Pipeline Steps](#pipeline_steps)
  - [Assign WCS Step](#wcs_step)
  - [Flat Field Step](#ff_step)
  - [Compare Rate vs. Flat Fielded Data](#compare)

**Author**: Rachel Plesha (rplesha@stsci.edu), Camilla Pacifici (cpacifici@stsci.edu)<br>
**Last modified**: March 2024

This notebook was last tested with JWST pipeline version 1.12.5 and the CRDS context jwst_1215.pmap.

<a id='imports'></a>
## Imports & Data Setup

In [ ]:
import os
import glob
import warnings
import numpy as np
import pandas as pd

import astropy.units as u
from astropy.io import ascii, fits
from astropy.table import Table
from astropy.coordinates import SkyCoord, Angle

from jdaviz import Imviz
from matplotlib import pyplot as plt
%matplotlib inline

import jwst
from jwst.assign_wcs import AssignWcsStep
from jwst.flatfield import FlatFieldStep

Check what version of the JWST pipeline you are using. To see what the latest version of the pipeline is available or install a previous version, check [GitHub](https://github.com/spacetelescope/jwst#software-vs-dms-build-version-map"). Also verify what [CRDS version](https://jwst-crds.stsci.edu/) you are using. [CRDS documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/user_documentation/reference_files_crds.html) explains how to set a specific context to use in the JWST pipeline. If either of these values are different from the last tested note above there may be differences in this notebook.

In [ ]:
import jwst
import crds
print('JWST Pipeliene Version:', jwst.__version__)
print('CRDS Context:', crds.get_context_name('jwst'))

#### Data setup
Open the file that contains all of the rate files, and create an output directory for the calibrated files if it does not already exist

In [ ]:
# From the csv file we created earlier, find a list of all of the grism observations we will want to calibrate with spec2
listrate_file = 'list_ngdeep_rate.csv'
rate_df = pd.read_csv(listrate_file)

In [ ]:
all_rate_files = rate_df['FILENAME']
data_dir_out = 'data/calibrated_steps/' # directory where to save the calibrate files

# if the directory does not exist that you want to save out to, make that directory first
if not os.path.exists(data_dir_out):
    os.makedirs(data_dir_out)

<a id='pipeline_steps'></a>
## Running Individual Pipeline Steps

While you could look at the rate images, instead consider running the files through the `assign_wcs` and `flat_field` steps of the pipeline to clean up detector artifacts.

<a id='wcs_step'></a>
#### Assign WCS Step

The `assign_wcs` step of the pipeline is a critical part to obtaining the correct spectral trace cutouts for WFSS images. To read more about the step, visit the [AssignWCS jdox page](https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/main.html).

In [ ]:
# Run assign_wcs
for ratefile in all_rate_files:
    result = AssignWcsStep.call(ratefile, output_dir=data_dir_out, save_results=True)

In [ ]:
# A quick sanity check to ensure that the files were calibrated.
# if this is zero, check the log message above for any errors that may have occurred during the calibration
wcsstep_files = glob.glob(os.path.join(data_dir_out, '*assignwcsstep*'))
print(len(wcsstep_files), 'assignwcsstep files written') # 29 files should have been written

<a id='ff_step'></a>
#### Flat Field Step

After the assignwcs file is run, we then want to run the `flat_field` step of the pipeline which removes detector artifacts using the flat field reference files. Read more about the step on [jdox](https://jwst-pipeline.readthedocs.io/en/latest/jwst/flatfield/main.html)

In [ ]:
# Run flat_field
for wcsfile in wcsstep_files:
    result = FlatFieldStep.call(wcsfile, output_dir=data_dir_out, save_results=True)

In [ ]:
# A quick sanity check to ensure that the files were calibrated.
# if this is zero, check the log message above for any errors that may have occurred during the calibration
flatfield_files = glob.glob(os.path.join(data_dir_out, '*flatfieldstep*'))
print(len(flatfield_files), 'flatfieldstep files written') # 29 files should have been written (matching the wcs step)

<a id='compare'></a>
#### Compare Rate vs. Flat fielded Data
Running the cell below shows the same direct image from just the rate file versus when the `flat_field` step of the pipeline is run. Some detector artifacts are noticably gone.

There are remaining optical artifacts to be aware of: https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-instrument-features-and-caveats

In [ ]:
test_rate_file = np.sort(list(rate_df[rate_df['FILTER'] == 'CLEAR']['FILENAME']))[0]
test_flat_file = os.path.join(data_dir_out, os.path.basename(test_rate_file).replace('rate.fits', 'flatfieldstep.fits'))

plot_files = [test_rate_file, test_flat_file]
plot_titles = ['Rate File', 'Flat Corrected File']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
fig.suptitle(f"{os.path.basename(test_rate_file).split('_rate')[0]}\n{fits.getval(test_rate_file, 'PUPIL')}", x=0.5, y=0.72)

for filename, title, ax in zip(plot_files, plot_titles, [ax1, ax2]):
    with fits.open(filename) as hdu:
        # fill in the nan values from the bad pixels with zero; otherwise a single, non-dithered image is impossible to really see
        data = hdu[1].data
        data[np.isnan(data)] = 0
        
        ax.imshow(data, vmin=0, vmax=1.7, origin='lower')
        ax.set_title(title)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>